# Summary

This Python notebook contains all the necessary and sufficient code to reproduce the results that were obtained for the original article. The resulting files are all saved and available on github.

1. [Data pre-processing](#preprocessing)
2. [PCA and LR](#pca-and-lr)
3. [PCA and SVM](#pca-and-svm)
3. [PCA and RF](#pca-and-rf)
4. [PCA and MLP](#pca-and-mlp)
5. [Probability map (biomarker) for AD](#probability-map)

## Data pre-processing <a name="preprocessing"></a>

All cells below contain the code related to pre-processing the neuroimages before being used for training/testing the classification models. The function below, for example, is responsible for loading neuroimages as *numpy arrays*, flatten or not.

In [ ]:
import nibabel as nib
import numpy as np
import os

# load neuroimages from specified path: returns an array containing all the images
# from the given path. The output can be flatten or not
def load_imgs_from_path(path, vectorize = True):

    # listing img files
    all_imgs_list = os.listdir(path)
    all_imgs_list = [file for file in all_imgs_list if file[-3:] == "img"]

    # loading imgs
    all_imgs = []
    for img_file in all_imgs_list:
        all_imgs.append(nib.load(path + img_file).get_fdata())

    # returning final list as an array (vectorized or not)
    if vectorize:
        all_imgs = np.array(all_imgs)
        return all_imgs.reshape((all_imgs.shape[0], np.prod(all_imgs.shape[1:])))
    else:
        return np.array(all_imgs)

To load the neuroimages, simply use the function above over the appropriate directories. Next, the neuroimages must be masked and passed through the [`scale`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html) function:



In [ ]:
from sklearn.pre_processing import scale

# loading ADNI data
ADNI_ad = load_imgs_from_path("./ADNI_AD/")
ADNI_cn = load_imgs_from_path("./ADNI_CN/")

# loading CDI data
clinic_ad = load_imgs_from_path("./CDI_AD/")
clinic_nad = load_imgs_from_path("./CDI_nAD/")

# loading mask
mask = load_imgs_from_path("./mask/").astype(int)
# flattening mask
mask = mask.reshape((mask.size, ))

## concatenating adni and CDI data separately
# and then masking and scaling it

# ADNI data
adni_data = np.concatenate((ADNI_ad, ADNI_cn))
adni_dataM = adni_data[:, mask == 1]
adni_dataMS = scale(adni_dataM, axis = 1)

# clinic data
clinic_data = np.concatenate((clinic_ad, clinic_nad))
clinic_dataM = clinic_data[:, mask == 1]
clinic_dataMS = scale(clinic_dataM, axis = 1)

# generating respective labels (AD as 1)
adni_labels = [1]*100 + [0]*100
clinic_labels = [1]*92 + [0]*100

# deleting some of the variables so we can use less RAM
del ADNI_ad, ADNI_cn, clinic_ad, clinic_nad

Organizing and saving training and testing data:

In [ ]:
import joblib as jb

# training data
X_train = adni_dataMS
y_train = adni_labels
jb.dump((X_train, y_train), "training_data.pkl")

# testing data
X_test = clinic_dataMS
y_test = clinic_labels
jb.dump((X_test, y_test), "testing_data.pkl")

## PCA and LR <a name="pca-and-lr"></a>

All code related to PCA and LR modeling is shown below. The `GridSearchCV` object was saved so it could be used later.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import joblib as jb

# loading training data
X_train, y_train = jb.load("training_data.pkl")

# loading testing data
X_test, y_test = jb.load("testing_data.pkl")

# pipeline with PCA and Logistic Regression
pipe = Pipeline([("PCA", PCA(n_components = .8, whiten = True)),
                 ("LR", LogisticRegression(solver = "saga", n_jobs = -1, max_iter = 5000))])

# param_grid for GridSearch
param_grid = [{"LR__penalty": ["l1", "l2"],
              "LR__C": np.logspace(-2, 2, 25)},
              {"LR__penalty": ["none"]}]

# fitting GridSearch and saving the results
gs = GridSearchCV(pipe, param_grid, scoring = ["accuracy", "recall", "precision", "f1"], refit = "f1")
gs.fit(X_train, y_train)
jb.dump(gs, "gs_fitted_LR.pkl")

# saving gs results as a spreadsheet
pd.DataFrame(gs.cv_results_).to_excel("params_tbl_LR.xlsx", index=False)

## PCA and SVM <a name="pca-and-svm"></a>

All code related to PCA and SVM modeling is shown below. The `GridSearchCV` object was saved so it could be used later.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import joblib as jb

# loading training data
X_train, y_train = jb.load("training_data.pkl")

# loading testing data
X_test, y_test = jb.load("testing_data.pkl")

# pipeline with PCA and SVC
pipe = Pipeline([("PCA", PCA(n_components = .8, whiten = True)),
                 ("SVC", SVC(probability=True))])

# param_grid for GridSearch
param_grid = [{"SVC__kernel": ["poly", "rbf", "sigmoid"],
             "SVC__C": np.logspace(-3, 2, 8),
             "SVC__gamma": np.logspace(-3, 2, 8)},
             {"SVC__kernel": ["linear"],
             "SVC__C": np.logspace(-3, 2, 8)}]

# fitting GridSearch and saving the results
gs = GridSearchCV(pipe, param_grid, scoring = ["accuracy", "recall", "precision", "f1"], refit = "f1")
gs.fit(X_train, y_train)
jb.dump(gs, "gs_fitted_SVC.pkl")

# saving gs results as a spreadsheet
pd.DataFrame(gs.cv_results_).to_excel("params_tbl_SVC.xlsx", index=False)

## PCA and RF <a name="pca-and-rf"></a>

All code related to PCA and RF modeling is shown below. The `GridSearchCV` object was saved so it could be used later.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import joblib as jb

# loading training data
X_train, y_train = jb.load("training_data.pkl")

# loading testing data
X_test, y_test = jb.load("testing_data.pkl")

# pipeline with PCA and Random Forest
pipe = Pipeline([("PCA", PCA(n_components = .8, whiten = True)),
                 ("RF", RandomForestClassifier(n_jobs = -1))])

# param_grid for GridSearch
param_grid = {"RF__n_estimators": [50, 100, 150],
             "RF__max_depth": list(range(3,10)) + ["None"],
             "RF__max_features": range(5, 16)}

# fitting GridSearch and saving the results
gs = GridSearchCV(pipe, param_grid, scoring = ["accuracy", "recall", "precision", "f1"], refit = "f1")
gs.fit(X_train, y_train)
jb.dump(gs, "gs_fitted_RF.pkl")

# saving gs results as a spreadsheet
pd.DataFrame(gs.cv_results_).to_excel("params_tbl_RF.xlsx", index=False)

## PCA and MLP <a name="pca-and-mlp"></a>

All code related to PCA and MLP modeling is shown below. The `GridSearchCV` object was saved so it could be used later.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import joblib as jb

# pipeline with PCA and Multilayer Perceptron
pipe = Pipeline([("PCA", PCA(n_components = .8, whiten = True)),
                 ("MLP", MLPClassifier())])

# param_grid for GridSearch
param_grid = {"MLP__alpha": np.logspace(-4, 2, 30),
             "MLP__hidden_layer_sizes": [(80,), (80, 70), (80, 70, 60)],
             "MLP__activation": ["logistic", "tanh", "relu"]}

# fitting GridSearch and saving the results
gs = GridSearchCV(pipe, param_grid, scoring = ["accuracy", "recall", "precision", "f1"], refit = "f1")
gs.fit(X_train, y_train)
jb.dump(gs, "gs_fitted_MLP.pkl")

# saving gs results as a spreadsheet
pd.DataFrame(gs.cv_results_).to_excel("params_tbl_MLP.xlsx", index=False)

## Probability map (biomarker) for AD <a name="probability-map"></a>

The code below refers to calculating the probability map (or biomarker) for Alzheimer's disease. The resulting .img/.hdr file can easily be opened using SPM or MRIcroGL for further visual processing.

In [ ]:
import nibabel as nib
import joblib as jb
import numpy as np

# loading mask
mask = load_imgs_from_path("./mask/").astype(int)
# flattening mask
mask = mask.reshape((mask.size, ))

# loading gs_LR
gs = jb.load("./gs_fitted_LR.pkl")
# separating classifier
clf = gs.best_estimator_["LR"]
# separating pca
pca = gs.best_estimator_["PCA"]

# initializing variable using the mask
biomarker = mask.copy().astype(float)

# for values ​​other than 0, enter the product between the principal components
# and their respective weights of the logistic regression model
biomarker[biomarker != 0] = np.sum(pca.components_ * clf.coef_.T, axis = 0)

# reshaping so that data can be saved using nib
biomarker = biomarker.reshape((79, 95, 79, 1))

# loading mask_file using nib so we can use affine and header attributes
# for the biomarker data (it could be any neuroimage in place of the mask)
mask_file = nib.load("./mask/OKmask20.img")
biomarker = nib.Nifti1Image(biomarker, affine = mask_file.affine, header = mask_file.header)

# saving object as .img/.hdr file for further image processing
nib.save(biomarker, "biomarker.img")